In [1]:
from tsrrt.collision import PandaCollision
import pybullet as p
import pybullet_data
import numpy as np
from spatial_math_mini import *
from collections.abc import Sequence
from abc import ABC, abstractmethod
import kdtree
import datetime

pybullet build time: Oct 11 2021 20:59:39


In [2]:
def view_pose(pos, ori):
    T = SE3.qtn_trans(ori, pos).T
    length = 0.1
    xaxis = np.array([length, 0, 0, 1])
    yaxis = np.array([0, length, 0, 1])
    zaxis = np.array([0, 0, length, 1])
    T_axis = np.array([xaxis, yaxis, zaxis]).T
    axes = T @ T_axis
    orig = T[:3,-1]
    xaxis = axes[:-1,0]
    yaxis = axes[:-1,1]
    zaxis = axes[:-1,2]
    x = p.addUserDebugLine(orig,xaxis, lineColorRGB=[1,0,0], lineWidth=5)
    y = p.addUserDebugLine(orig,yaxis, lineColorRGB=[0,1,0], lineWidth=5)
    z = p.addUserDebugLine(orig,zaxis, lineColorRGB=[0,0,1], lineWidth=5)
    pose_id = [x, y, z]
    return pose_id

def view_node(node):
    view_pose(node.pos, [1,0,0,0])

def clear(rng=None):
    if rng is None:
        rng = range(100)
    for i in rng:
        p.removeUserDebugItem(i)

In [3]:
class NodeTaskPosOri(Sequence):
    def __init__(self, pos, ori, q, m=None):
        super().__init__()
        self.pos = pos
        self.ori = ori
        self.q = q
        self._m = m
        self.idx = None

    def __getitem__(self, i):
        return self.pos[i]

    def __len__(self):
        return len(self.pos)
    
    def __repr__(self):
        return "pos:{}, ori:{}".format(self.pos, self.ori)
    
    def __sub__(self, y):
        """get twist"""
        SE3_1 = SE3(self.ori, self.pos)
        SE3_2 = SE3(y.ori, y.pos)
        screw, angle = (SE3_2.inv() @ SE3_1).to_twistangle()
        return screw*angle

class NodeTS(Sequence):
    def __init__(self, pos, ori, q, m=None):
        super().__init__()
        self.pos = pos
        self.ori = ori
        self.q = q
        self._m = m
        self.idx = None
        
    def __repr__(self):
        return "pos:{}, ori:{}".format(self.pos, self.ori)
    
    def __sub__(self, y):
        """get twist"""
        SE3_1 = SE3(self.ori, self.pos)
        SE3_2 = SE3(y.ori, y.pos)
        screw, angle = (SE3_2.inv() @ SE3_1).to_twistangle()
        return screw*angle

class Tree:
    def __init__(self, root):
        #init
        self._kdtree = kdtree.create(dimensions=len(root))
        self._parent_of = {0:None} # parent of 0(root) is None
        root.idx = 0
        self._data = [root]
        self._kdtree.add(root)
        self._num = 1
        
        #config
        self._rebalance_term = 20
    
    def add(self, parent, child):
        child.idx = self._num
        self._data.append(child)
        self._kdtree.add(child)
        self._parent_of[child.idx] = parent.idx
        self._num += 1
        
        if self._num % self._rebalance_term == 0:
            root = self._kdtree.rebalance()
    
    def nearest(self, node):
        node, dist = self._kdtree.search_nn(node)
        return node.data
    
    def parent(self, node):
        assert node.idx is not None
        return self._data[self._parent_of[node.idx]]
    
    def backtrack(self, node_last):
        path = [node_last]
        child_idx = node_last.idx
        while True:
            parent_idx = self._parent_of[child_idx]
            if parent_idx is None:
                break
            path.append(self._data[parent_idx])
            child_idx = parent_idx
        return path[::-1]

In [4]:
class StateSpace:
    """ 
    state space configuration and random node generator
    """
    pos_ll = [-1, -1, 0]
    pos_ul = [1, 1, 1]
    q_ll = [-2.9671, -1.8326, -2.9671, -3.1416, -2.9671, -0.0873, -2.9671]
    q_ul = [2.9671, 1.8326, 2.9671, 0.0, 2.9671, 3.8223, 2.9671]
    q_m = [0.0, 0.0, 0.0, -1.5708, 0.0, 1.8675, 0.0]
    q_range = [5.9342, 3.6652, 5.9342, 3.1416, 5.9342, 3.9095999999999997, 5.9342]
    eps = 0.01
    trans_max = 0.1
    rot_max = 0.1
    
    def __init__(self, node_type, checker):
        self._node_class = node_type
        self._checker = checker
        
    @staticmethod
    def _get_pos():
        pos = np.empty(3)
        for i in range(3):
            ll, ul = StateSpace.pos_ll[i], StateSpace.pos_ul[i]
            pos[i] = np.random.uniform(low=ll, high=ul)
        return pos
    
    @staticmethod
    def _get_ori():
        return  SO3._uniform_sampling_quaternion()
    
    @staticmethod
    def _get_q():
        q = np.empty(7)
        for i in range(7):
            ll, ul = StateSpace.q_ll[i], StateSpace.q_ul[i]
            q[i] = np.random.uniform(low=ll, high=ul)
        return q
    
    def is_valid(self, q):
        checker.set_joint_positions(q)
        return not checker.is_self_collision()

    def random(self):
        #get sample in free space by rejection sampling
        while True:
            #task-space voronoi bias
            pos, ori = self._get_pos(), self._get_ori()
            _, q = self._checker.IK(pos, ori)
            self._checker.set_joint_positions(q)
            pos, ori = self._checker.get_ee_pose()
            is_collision = self._checker.is_self_collision()
            if not is_collision:
                node_new = self._node_class(pos, ori, q)
                break
        return node_new

In [27]:
class Controller:
    trans_max = 0.1
    rot_max = 0.1
    manipulability_max = 1000
    
    def __init__(self, node_type, checker):
        self._node_class = node_type
        self._checker = checker
        
    @staticmethod
    def crop_within_limits(twist):
        trans_ratio = np.linalg.norm(twist[:3])/Controller.trans_max
        rot_ratio = np.linalg.norm(twist[3:])/Controller.rot_max
        if (trans_ratio < 1) & (rot_ratio < 1):
            return twist
        elif trans_ratio > rot_ratio:
            return twist/trans_ratio
        else:
            return twist/rot_ratio
    
    def clamp_max(self, qs, qmax):
        result = []
        mag = np.linalg.norm(qs)
        if mag < qmax:
            return qs
        else:
            return qmax * qs/mag
    
    def clamp_max_abs(self, qs, qmax):
        result = []
        mag = np.linalg.norm(qs,1)
        if mag < qmax:
            return qs
        else:
            return qmax * qs/mag
        
    def control(self, node1, node2):
        diff = node2 - node1
        jac = self._checker.get_body_jacobian(node1.q)
        #jac_pinv = np.linalg.pinv(jac)
        #null_proj = np.eye(7) - jac_pinv@jac
        #damped least square
        U, s, VT = np.linalg.svd(jac)
        damping = 0.1
        dls = np.zeros((7,6))
        for i in range(6):
            u, v = U[:,i], VT[i,:]
            dls += s[i]/(s[i]**2+damping**2) * np.outer(v, u)
        manipulability = s[0] / s[-1]
        # print(np.linalg.norm(null_proj))
        # print(null_proj @ (node2.q - node1.q))
        q_delta = dls @ diff  # + null_proj @ (StateSpace.q_m - node1.q)
        q_new = self.clamp_max(q_delta, 0.1) + node1.q
        pos, ori = self._checker.FK(q_new)
        is_stop_cond = manipulability > self.manipulability_max
        return False, self._node_class(pos, ori, q_new, manipulability)

In [6]:
gui_uid = p.connect(p.GUI)
checker = PandaCollision(p, gui_uid)

In [2]:
_statespace = StateSpace(NodeTaskPosOri, checker)
_controller = Controller(NodeTaskPosOri, checker)

#init
checker.set_home_positions()
(pos, ori), q = checker.get_ee_pose(), checker.get_arm_positions()
start = NodeTaskPosOri(pos, ori, q)
_tree = Tree(start)

NameError: name 'StateSpace' is not defined

In [1]:
pos, ori = start.pos+np.array([-0.1, -0.2, 0.1]), start.ori
success, q = checker.IK(pos, ori)
goal = NodeTaskPosOri(pos, ori, q)
checker.set_joint_positions(goal.q)
view_pose(goal.pos, goal.ori)

NameError: name 'start' is not defined

In [16]:
goal = _statespace.random()
view_pose(goal.pos, goal.ori)
checker.set_joint_positions(goal.q)
checker.is_self_collision()

False

In [30]:
is_stop, node = _controller.control(start, goal)
checker.set_joint_positions(node.q)

In [35]:
if not is_stop:
    success, node = _controller.control(node, goal)
    print(np.linalg.norm(goal-node))
    print(np.linalg.norm(goal.q - node.q))
checker.set_joint_positions(node.q)

0.006541100680427883
3.1228390351776416


0.0036104007871009544

In [35]:
diff = goal - node
jac = checker.get_body_jacobian(node.q)
jac_pinv = np.linalg.pinv(jac)

In [36]:
jac_pinv@diff

array([-4.60103567, -3.20814975,  2.86952484, -9.7671622 ,  0.79606819,
        4.64126929, -4.30075631])

In [39]:
U, s, VT = np.linalg.svd(jac)
V = VT.T

In [47]:
damping = 0.1
Jpinv = np.zeros((7,6))
for i in range(6):
    u, v = U[:,i], VT[i,:]
    Jpinv += s[i]/(s[i]**2+damping**2) * np.outer(v, u)

In [48]:
Jpinv@diff

array([-0.4601617 , -0.07196402,  1.80160858, -0.22469696,  0.55231929,
        0.20198507, -2.40271845])

In [42]:
jac_pinv

array([[  2.97890135,  -3.7526373 ,   2.00694745,   1.94803831,
         -5.93304865, -13.45082458],
       [  2.52789543,  -1.68936048,   0.8563515 ,  -0.04799538,
         -5.92050149, -12.74544068],
       [ -0.49902649,   3.50054497,  -1.09248351,  -1.91090129,
         -0.62736833,  -0.61188945],
       [  9.94389686,  -6.83738996,   3.15529492,   2.12306964,
        -12.95038206, -34.7475844 ],
       [ -0.09956965,   1.08110574,  -0.0908838 ,  -0.59700884,
         -0.33088829,  -0.48719466],
       [ -7.03989854,   2.80715606,  -1.11804916,  -0.75188744,
          4.58937498,  16.25930769],
       [  0.89240757,  -5.06939729,   2.58071469,   3.08519723,
          0.4738041 ,  -0.13950754]])

In [4207]:
np.linalg.norm(checker.get_body_jacobian())

4.1516493044891245

array([ 2.45213507, -2.52519881, -1.64474527, -4.73409451, -1.55111872,
       -0.56737927,  6.15253383])

In [4079]:
goal.q

array([-0.17950216,  1.26708591, -0.12268157, -0.77751842, -0.06341397,
        4.25270392,  0.20469272])

In [4211]:
checker.set_joint_positions(goal.q)

In [2692]:
goal.q

array([ 0.56273421,  1.08908057,  0.26057273, -0.35235054,  0.04545562,
        3.87986696, -0.22801552])

In [ ]:
_last_node = None
_start = start
_goal = goal
_tree = Tree(start)
_controller = Controller(checker)
_statespace = StateSpace(checker)

In [20]:
class NodeTaskPosOri2(Sequence):
    def __init__(self, pos, ori, q, m=None):
        super().__init__()
        self.pos = pos
        self.ori = ori
        self.q = q
        self._m = m
        self.idx = None

    def __getitem__(self, i):
        return self.q[i]

    def __len__(self):
        return len(self.q)
    
    def __repr__(self):
        return "pos:{}, ori:{}".format(self.pos, self.ori)
    
    def __sub__(self, y):
        SE3_1 = SE3(self.ori, self.pos)
        SE3_2 = SE3(y.ori, y.pos)
        screw, angle = (SE3_2.inv() @ SE3_1).to_twistangle()
        return screw*angle
        
    @staticmethod
    def distance(node1, node2):
        """define distance between two nodes"""
        trans_dist = np.linalg.norm(node1.pos - node2.pos)
        q1, q2 = node1.ori, node2.ori
        lmda = q1 @ q2
        if lmda < 0:
            q2, lmda = -q2, -lmda
        if np.abs(1 - lmda) < 0.001:
            rot_dist = 0
        else:
            alpha = np.arccos(lmda)
            rot_dist = 2*alpha
        return np.linalg.norm([trans_dist, rot_dist])

class Tree:
    def __init__(self, root):
        #init
        self._kdtree = kdtree.create(dimensions=len(root))
        self._parent_of = {0:None} # parent of 0(root) is None
        root.idx = 0
        self._data = [root]
        self._kdtree.add(root)
        self._num = 1
        
        #config
        self._rebalance_term = 20
    
    def add(self, parent, child):
        child.idx = self._num
        self._data.append(child)
        self._kdtree.add(child)
        self._parent_of[child.idx] = parent.idx
        self._num += 1
        
        if self._num % self._rebalance_term == 0:
            root = self._kdtree.rebalance()
    
    def nearest(self, node):
        node, dist = self._kdtree.search_nn(node)
        return node.data
    
    def parent(self, node):
        assert node.idx is not None
        return self._data[self._parent_of[node.idx]]

class StateSpace:
    """ 
    state space configuration and random node generator
    """
    pos_ll = [-1, -1, 0]
    pos_ul = [1, 1, 1]
    q_ll = [-2.9671, -1.8326, -2.9671, -3.1416, -2.9671, -0.0873, -2.9671]
    q_ul = [2.9671, 1.8326, 2.9671, 0.0, 2.9671, 3.8223, 2.9671]
    q_m = [0.0, 0.0, 0.0, -1.5708, 0.0, 1.8675, 0.0]
    q_range = [5.9342, 3.6652, 5.9342, 3.1416, 5.9342, 3.9095999999999997, 5.9342]
    eps = 0.01
    trans_max = 0.1
    rot_max = 0.1
    
    def __init__(self, checker):
        self._checker = checker
        
    @staticmethod
    def _get_pos():
        pos = np.empty(3)
        for i in range(3):
            ll, ul = StateSpace.pos_ll[i], StateSpace.pos_ul[i]
            pos[i] = np.random.uniform(low=ll, high=ul)
        return pos
    
    @staticmethod
    def _get_ori():
        return  SO3._uniform_sampling_quaternion()
    
    @staticmethod
    def _get_q():
        q = np.empty(7)
        for i in range(7):
            ll, ul = StateSpace.q_ll[i], StateSpace.q_ul[i]
            q[i] = np.random.uniform(low=ll, high=ul)
        return q
    
    def is_valid(self, node):
        checker.set_joint_positions(node.q)
        return not checker.is_self_collision()

    def random(self):
        #get sample in free space by rejection sampling
        while True:
            #task-space voronoi bias
            pos = self._get_pos()
            ori = self._get_ori()
            _, q = self._checker.IK(pos, ori)
            self._checker.set_joint_positions(q)
            pos, ori = self._checker.get_ee_pose()
            is_collision = self._checker.is_self_collision()
            if (not is_collision):
                node_new = NodeTaskPosOri2(pos, ori, q)
                break
        return node_new
    
class Controller:
    trans_max = 0.1
    rot_max = 0.1
    manipulability_max = 1000
    
    def __init__(self, checker):
        self._checker = checker
        
    @staticmethod
    def crop_within_limits(twist):
        trans_ratio = np.linalg.norm(twist[:3])/Controller.trans_max
        rot_ratio = np.linalg.norm(twist[3:])/Controller.rot_max
        if (trans_ratio < 1) & (rot_ratio < 1):
            return twist
        elif trans_ratio > rot_ratio:
            return twist/trans_ratio
        else:
            return twist/rot_ratio
    
    def clamp_mag(self, qs):
        d_max = 0.2
        result = []
        mag = np.linalg.norm(qs)
        if mag < d_max:
            return qs
        else:
            return d_max * qs/mag
        
    def control(self, node1, node2):
        diff = node2 - node1
        #diff = Controller.crop_within_limits(diff)
        jac = self._checker.get_body_jacobian(node1.q)
        jac_pinv = np.linalg.pinv(jac)
        null_proj = np.eye(7) - jac_pinv@jac
        _, s, _ = np.linalg.svd(jac)
        manipulability = s[0] / s[-1]
        q_delta = jac_pinv @ diff #+ null_proj @ (node2.q - node1.q)*0.1
        q_new = self.clamp_mag(q_delta) + node1.q
        pos, ori = self._checker.FK(q_new)
        is_stop_cond = manipulability > self.manipulability_max
        return False, NodeTaskPosOri2(pos, ori, q_new, manipulability)
    



In [21]:
class RRT:
    def __init__(self, start, goal, checker):
        self._checker = checker
        self._controller = Controller(self._checker)
        self._statespace = StateSpace(self._checker)
        self._tree = Tree(start)
        
        self._start = start
        self._goal = goal
        self._last_node = None
        self._node_class = type(start)
        
        self._goal_bias = 0.05
        
    def extend(self, node_rand):
        node_near = self._tree.nearest(node_rand)
        
        parent = node_near
        is_stop, node_new = self._controller.control(parent, node_rand)
        
        while True:
            is_not_valid = not self._statespace.is_valid(node_new)
            is_goal = self._node_class.distance(self._goal, node_new) <= self._statespace.eps
            if is_not_valid | is_stop:
                break
            if is_goal:
                self._last_node = node_new
                break
                
            #self._checker.set_joint_positions(node_new.q)#######################################################
            self._tree.add(parent, node_new)
            
            parent = node_new
            is_stop, node_new = self._controller.control(parent, node_rand)
    
    def is_goal_found(self):
        return self._last_node is not None
    
    def plan(self, max_time=5000000000000000000000000.0):
        self._last_node = None
        start_time = datetime.datetime.now()
        while not self.is_goal_found():
            # if np.random.random() > self._goal_bias:
            #     node_rand = self._statespace.random()
            # else:
            node_rand = self._goal
            view_pose(node_rand.pos, node_rand.ori)##########################################3
            self.extend(node_rand)

            time_delta = (datetime.datetime.now() - start_time)
            elapsed = time_delta.seconds + 0.000001 * time_delta.microseconds
            if elapsed > max_time:
                break

        if self.is_goal_found():
            print("planning success")
        else:
            print("planning fail")
        print("elapsed time : {}".format(elapsed))
        print("nodes:{}".format(self._tree._num))
        return self.is_goal_found(), self.path()

    def path(self):
        if not self.is_goal_found():
            return []
        path = [self._last_node]
        child_idx = self._last_node.idx
        while True:
            parent_idx = self._tree._parent_of[child_idx]
            if parent_idx is None:
                break
            path.append(self._tree._data[parent_idx])
            child_idx = parent_idx
        return path[::-1]

    def reset(self):
        self._tree = Tree(self._start)
        self._last_node = None

In [22]:
ss = StateSpace(checker)
checker.set_home_positions()
pos, ori = checker.get_ee_pose()
q = checker.get_arm_positions()
start = NodeTaskPosOri2(pos, ori, q)
goal = ss.random()

In [23]:
goal = ss.random()
view_pose(goal.pos, goal.ori)
checker.set_joint_positions(goal.q)
checker.is_self_collision()

False

In [31]:
import copy
def extend(node_rand):
    global _last_node
    node_near = _tree.nearest(node_rand)
    #view_pose(node_rand.pos, node_rand.ori)
    parent = node_near
    is_stop, node_new = _controller.control(parent, node_rand)

    #while True:
    is_not_valid = not _statespace.is_valid(node_new)
    #is_arrived = NodeTaskPosOri2.distance(node_rand, node_new) <= _statespace.eps
    is_goal = NodeTaskPosOri2.distance(_goal, node_new) <= _statespace.eps
    print(is_goal, NodeTaskPosOri2.distance(_goal, node_new), is_not_valid, is_stop)
    
    if is_not_valid | is_stop:
        return #break
        
    checker.set_joint_positions(node_new.q)#######################################################
    _tree.add(parent, node_new)
    
    if is_goal:
        print(node_new)
        _last_node = copy.deepcopy(node_new)
        return #break

    #parent = node_new
    #is_stop, node_new = _controller.control(parent, node_rand)

def is_goal():
    return _last_node is not None

global _last_node
_last_node = None
_start = start
_goal = goal
_tree = Tree(start)
_controller = Controller(checker)
_statespace = StateSpace(checker)


In [32]:
while _last_node is None:
    if np.random.random() > 0.05:
        node_rand = _statespace.random()
    else:
        node_rand = _goal
    #view_pose(node_rand.pos, node_rand.ori)##########################################3
    extend(node_rand)

False 1.819803473325885 False False
False 1.8239251504026555 False False
False 1.8820073855615638 False False
False 1.8872634966266182 False False
False 1.8071889077421535 False False
False 1.787863437138645 False False
False 1.839647089062256 False False
False 1.7516534148794667 False False
False 1.7004168785945708 False False
False 1.644283952941847 False False
False 1.7402878379512419 False False
False 1.6332016720268767 False False
False 1.6155098188086858 False False
False 1.5922522426131305 False False
False 1.7933082438352792 False False
False 1.6131145262080668 False False
False 1.605111376611624 False False
False 1.7731562498009272 False False
False 1.5188415167627416 False False
False 1.805596844916538 False False
False 1.8293440852936806 False False
False 1.4510075874371349 False False
False 1.4180743481538263 False False
False 1.7799360961728008 False False
False 1.7657520106486926 False False
False 1.785566422529369 False False
False 1.7615748611003008 False False
False 1.

KeyboardInterrupt: 

In [809]:
rrt = RRT(start, goal, checker)

In [219]:
rrt.plan()

KeyboardInterrupt: 

In [8]:
checker.is_self_collision()

True

In [82]:
_tree.nearest(goal).q

array([ 0.        ,  0.        ,  0.        , -1.57079633,  0.        ,
        1.57079633,  0.78539816])

In [76]:
_tree.nearest(goal).q

array([ 0.        ,  0.        ,  0.        , -1.57079633,  0.        ,
        1.57079633,  0.78539816])

In [ ]:
clear()

In [817]:
node = rrt._tree.nearest(goal)

In [592]:
node_rand = goal

In [593]:
node_near = _tree.nearest(node_rand)
parent = node_near
is_stop, node_new = _controller.control(parent, node_rand)

In [790]:
print("isvalid?:", _statespace.is_valid(node_new))
print("dist:", NodeTaskPosOri2.distance(goal, node_new), "eps:", _statespace.eps)
print("isstop?:",is_stop, "manipul", node_new._m)
parent = node_new
is_stop, node_new = _controller.control(parent, node_rand)

isvalid?: True
dist: 1.307017700716942e-07 eps: 0.01
isstop?: False manipul 36.19576442558914


In [ ]:
rrt._tree.add(parent, node_new)


In [129]:
rrt._checker.set_joint_positions(node_new.q)#######################################################


In [ ]:
# rrt
node_rand = self._goal
#view_pose(node_rand.pos, node_rand.ori)##########################################3

node_near = rrt._tree.nearest(node_rand)
if self._node_class.distance(node_rand, node_near) < self._statespace.eps:
    # same node
    return

parent = node_near
is_stop, node_new = self._controller.control(parent, node_rand)

while True:
    self._checker.set_joint_positions(node_new.q)#######################################################
    if (not self._statespace.is_valid(node_new)) | is_stop:
        break
    self._tree.add(parent, node_new)

    if (self._node_class.distance(self._goal, node_new) <= self._statespace.eps):
        self._last_node = node_new
        break
    parent = node_new
    is_stop, node_new = self._controller.control(parent, node_rand)

In [121]:
rrt.plan()

KeyboardInterrupt: 

In [76]:
nn = rrt._tree.nearest(goal)

In [83]:
checker.set_joint_positions(nn.q)

In [84]:
checker.set_joint_positions(goal)

In [85]:
_, node = rrt._controller.control(nn, goal)
checker.set_joint_positions(node.q)

In [104]:
_, node = rrt._controller.control(node, goal)
checker.set_joint_positions(node.q)
print(node._m)
print(NodeTaskPosOri2.distance(node, goal))
print(checker.is_self_collision())
rrt._node_class.distance(rrt._goal, node) <= rrt._statespace.eps

24.618272524295467
0.00024711205309240516
False


True

In [70]:
col_list = []
for link1, link2 in checker.panda_self_collision_list():
    if abs(link1 - link2) == 1:
        continue
    dist_info = checker._client.getClosestPoints(bodyA=checker._robot, bodyB=checker._robot, distance=0.0,
                                  linkIndexA=link1, linkIndexB=link2, physicsClientId=checker._uid)
    if len(dist_info) != 0:
        col_list.append((link1,link2, dist_info))


In [71]:
col_list

[(4,
  8,
  ((0,
    0,
    0,
    4,
    8,
    (-0.2594083115242081, -0.049250562540415384, 0.2990032111850073),
    (-0.2597281083685291, -0.04958570900948539, 0.29879136703453263),
    (-0.6278120678296034, -0.657945822509202, -0.4158837603588373),
    -0.0005093830792812989,
    0.0,
    0.0,
    (0.0, 0.0, 0.0),
    0.0,
    (0.0, 0.0, 0.0)),))]

In [68]:
col_list

[(4, 8)]

In [ ]:
checker.is_self_collision()

In [1410]:
jac = checker.get_space_jacobian(nn.q)
jac_pinv = np.linalg.pinv(jac)
null_proj = np.eye(7) - jac_pinv@jac

In [1888]:
checker.set_joint_positions(nn.q)

In [1889]:
q_null = null_proj @ (goal.q - nn.q)
checker.set_joint_positions(q_null + nn.q)

In [1409]:
q_null

array([-0.41408551,  0.88364057, -0.04454825,  1.50828597,  0.27478869,
        0.52491271, -0.34667615])

In [1058]:
jac@null_jac @np.random.random(7)

array([ 1.85076278e-16,  2.04892328e-16,  5.24337724e-16, -2.97379152e-16,
        1.02722889e-17,  8.29310372e-17])

In [775]:
checker.is_self_collision()

True

In [784]:
checker.set_joint_positions(goal.q)

In [385]:
node._m

33.98086015365212

In [163]:
clear()

In [429]:
ss = StateSpace(checker)
c = Controller(checker)

In [803]:
node_rand = ss.random()
view_pose(node_rand.pos, node_rand.ori)

[84, 85, 86]

In [804]:
is_stop_cond, node = c.control(root, node_rand)
checker.set_joint_positions(node.q)

In [844]:
if not is_stop_cond:
    is_stop_cond, node = c.control(node, node_rand)
checker.set_joint_positions(node.q)

In [591]:
view_pose(node_rand.pos, node_rand.ori)

[54, 55, 56]

In [592]:
checker.set_joint_positions(node.q)

In [594]:
for i in range(100):
    node_rand = ss.random()
    parent = tree.nearest(node_rand)
    tree.add(parent, node_rand)

In [595]:
node_rand = ss.random()
nn = tree.nearest(node_rand)

In [596]:
view_pose(node_rand.pos, node_rand.ori)
checker.set_joint_positions(nn.q)

In [597]:
checker.set_joint_positions(node_rand.q)

In [598]:
pos, ori = checker.get_ee_pose()
view_pose(pos, ori)

[60, 61, 62]

In [599]:
view_pose(node_rand.pos, node_rand.ori)
checker.set_joint_positions(node_rand.q)

In [600]:
view_pose(node_rand.pos, node_rand.ori)
checker.set_joint_positions(node_rand.q)